In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
import os
import random
import numpy as np
from tqdm import tqdm
from skimage import io
from skimage.transform import resize
import matplotlib.pyplot as plt
import random
import time
from sklearn.model_selection import train_test_split

In [3]:
seed = 42
np.random.seed = seed

IMG_WIDTH = 499
IMG_HEIGHT = 802
IMG_CHANNEL = 22

In [4]:
def get_data(file_path):
    
    file = open(file_path, "r")

    allText = file.read()

    words = list(map(str, allText.split()))

    matrice_height = int(len(words) / 30)
    matrice = np.ones((matrice_height, 24))

    emplacement = 20
    iter1 = 0
    for i in range(emplacement, int(len(words) / 30) + emplacement):
        iter2 = 0
        for j in range(emplacement + 6, emplacement + 30):
            matrice[(iter1, iter2)] = words[j]
            iter2 += 1
        iter1 += 1
        emplacement += 30
    X_2 = np.delete(matrice, 1, 1)
    X_final = np.delete(X_2, 10, 1)
    
    file.close()
    
    return X_final

In [5]:
file_in_path = "C_maps_in.txt"
file_out_path = "C_maps_out.txt"

X = get_data(file_in_path)
y = get_data(file_out_path)

In [6]:
X.shape

(400198, 22)

In [7]:
X = X.reshape((1, 499, 802, 22))

In [8]:
X.shape

(1, 499, 802, 22)

In [9]:
y = y.reshape((1, 499, 802, 22))

In [10]:
y.shape

(1, 499, 802, 22)

In [11]:
y.shape

(1, 499, 802, 22)

In [61]:
X1 = X[:, :32, :128, :]
X2 = X[:, :32, 134:262, :]
X3 = X[:, :32, 268:396, :]
X4 = X[:, :32, 402:530, :]
X5 = X[:, :32, 536:664, :]
X6 = X[:, :32, 670:798, :]

X7 = X[:, 33:65, :128 , :]
X8 = X[:, 33:65, 134:262, :]
X9 = X[:, 33:65, 268:396, :]
X10 = X[:, 33:65, 402:530, :]
X11 = X[:, 33:65, 536:664, :]
X12 = X[:, 33:65, 670:798, :]

X13 = X[:, 66:98, :128 , :]
X14 = X[:, 66:98, 134:262, :]
X15 = X[:, 66:98, 268:396, :]
X16 = X[:, 66:98, 402:530, :]
X17 = X[:, 66:98, 536:664, :]
X18 = X[:, 66:98, 670:798, :]

X13 = X[:, 99:131, :128 , :]
X14 = X[:, 99:131, 134:262, :]
X15 = X[:, 99:131, 268:396, :]
X16 = X[:, 99:131, 402:530, :]
X17 = X[:, 99:131, 536:664, :]
X18 = X[:, 99:131, 670:798, :]
#===============================
y1 = y[:, :160, :128, :]
y2 = y[:, :160, 134:262, :]
y3 = y[:, :160, 268:396, :]
y4 = y[:, :160, 402:530, :]
y5 = y[:, :160, 536:664, :]
y6 = y[:, :160, 670:798, :]

y7 = y[:, 166:326, :128 , :]
y8 = y[:, 166:326, 134:262, :]
y9 = y[:, 166:326, 268:396, :]
y10 = y[:, 166:326, 402:530, :]
y11 = y[:, 166:326, 536:664, :]
y12 = y[:, 166:326, 670:798, :]

y13 = y[:, 332:492, :128 , :]
y14 = y[:, 332:492, 134:262, :]
y15 = y[:, 332:492, 268:396, :]
y16 = y[:, 332:492, 402:530, :]
y17 = y[:, 332:492, 536:664, :]
y18 = y[:, 332:492, 670:798, :]

In [62]:
print(y1.shape)

(1, 160, 128, 22)


In [63]:
new_X = np.concatenate((X1, X2), axis=0)
new_Y = np.concatenate((y1, y2), axis=0)
new_X = np.concatenate((new_X, X3), axis=0)
new_Y = np.concatenate((new_Y, y3), axis=0)
new_X = np.concatenate((new_X, X4), axis=0)
new_Y = np.concatenate((new_Y, y4), axis=0)
new_X = np.concatenate((new_X, X5), axis=0)
new_Y = np.concatenate((new_Y, y5), axis=0)
new_X = np.concatenate((new_X, X6), axis=0)
new_Y = np.concatenate((new_Y, y6), axis=0)
new_X = np.concatenate((new_X, X7), axis=0)
new_Y = np.concatenate((new_Y, y7), axis=0)
new_X = np.concatenate((new_X, X8), axis=0)
new_Y = np.concatenate((new_Y, y8), axis=0)
new_X = np.concatenate((new_X, X9), axis=0)
new_Y = np.concatenate((new_Y, y9), axis=0)
new_X = np.concatenate((new_X, X10), axis=0)
new_Y = np.concatenate((new_Y, y10), axis=0)
new_X = np.concatenate((new_X, X11), axis=0)
new_Y = np.concatenate((new_Y, y11), axis=0)
new_X = np.concatenate((new_X, X12), axis=0)
new_Y = np.concatenate((new_Y, y12), axis=0)
new_X = np.concatenate((new_X, X13), axis=0)
new_Y = np.concatenate((new_Y, y13), axis=0)
new_X = np.concatenate((new_X, X14), axis=0)
new_Y = np.concatenate((new_Y, y14), axis=0)
new_X = np.concatenate((new_X, X15), axis=0)
new_Y = np.concatenate((new_Y, y15), axis=0)
new_X = np.concatenate((new_X, X16), axis=0)
new_Y = np.concatenate((new_Y, y16), axis=0)
new_X = np.concatenate((new_X, X17), axis=0)
new_Y = np.concatenate((new_Y, y17), axis=0)
new_X = np.concatenate((new_X, X18), axis=0)
new_Y = np.concatenate((new_Y, y18), axis=0)

In [64]:
new_X.shape

(18, 160, 128, 22)

séparer en 12 l'image, 166x200, 802

In [65]:
# Build the model
inputs = tf.keras.layers.Input((IMG_WIDTH, IMG_HEIGHT, IMG_CHANNEL))

In [66]:
# Contraction path
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(inputs)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

c3 = tf.keras.layers.Conv2D(64, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

c4 = tf.keras.layers.Conv2D(128, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(c4)
p4 = tf.keras.layers.MaxPooling2D((2, 2))(c4)

c5 = tf.keras.layers.Conv2D(256, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(c5)


In [67]:
# Expansive path
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding="same")(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(c6)

u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding="same")(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(c7)

u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding="same")(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(c8)

u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding="same")(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(c9)

outputs = tf.keras.layers.Conv2D(1, (1, 1))(c9) # activation="sigmoid"

model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer="adam", loss="mae", metrics=["accuracy"])
model.summary()

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 248, 400, 32), (None, 249, 401, 32)]

In [ ]:
y.shape

In [ ]:
X = X[:,:256,:512,:]

In [ ]:
y = y[:,:256,:512,:]

In [ ]:
y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
# Model checkpoint
checkpointer = tf.keras.callbacks.ModelCheckpoint("Unet.h5", verbose=1, save_best_only=True)
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=10, monitor="loss"),
    tf.keras.callbacks.TensorBoard(log_dir="logs")
]

results = model.fit(
    X_train,
    y_train,
    # validation_split=0.1,
    batch_size=16,
    epochs=100,
    callbacks=callbacks,
    validation_data=(X_test, y_test)
)

In [ ]:
y_pred.shape

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_test = y_test.reshape(5, 131072)
y_pred = y_pred.reshape(5, 131072)

In [ ]:
y_test = y_test.T

In [ ]:
y_pred = y_pred.T

In [ ]:
y_pred.shape

In [ ]:
print("y_test : \n", y_test[0,0,1])
print("=====================================")
print("y_pred : \n", y_pred[0,0,1])

In [ ]:
axe = plt.axes()
plt.rcParams.update({'font.size':15})
plt.style.use('seaborn-whitegrid')
plt.scatter(y_test[:,0], y_pred[:,0], s=1)
plt.title('Coefficients')
axe.set(xlabel="y_test", ylabel="y_pred")
plt.show()
# plt.scatter(X_test, y_test, label="Données d'évaluation", color="b")

In [ ]:
idx = random.randint(0, len(X_train))
preds_train = model.predict(X_train[:int(X_train.shape[0]*0.9)], verbose=1)
preds_val = model.predict(X_train[int(X_train.shape[0]*0.9):], verbose=1)
preds_test = model.predict(X_test, verbose = 1)

In [ ]:
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

In [ ]:
ix = random.randint(0, len(preds_train_t))
imshow(X_train[ix])
plt.show()
imshow(np.squeeze(y_train[ix]))
plt.show()
imshow(np.squeeze(preds_train_t[ix]))
plt.show()

In [ ]:
ix = random.randint(0, len(preds_val_t))
imshow(X_train[int(X_train.shape[0]*0.9):][ix])
plt.show()
imshow(np.squeeze(y_train[int(y_train.shape[0]*0.9):][ix]))
plt.show()
imshow(np.squeeze(preds_val_t[ix]))
plt.show()
